In [16]:
! pip3 list

Package                  Version
------------------------ ----------
allennlp                 2.10.1
altgraph                 0.17.2
annotated-types          0.6.0
anyio                    4.2.0
appnope                  0.1.4
asttokens                2.4.1
backoff                  2.2.1
base58                   2.1.1
blis                     0.7.11
boto3                    1.34.39
botocore                 1.34.39
cached-path              1.1.6
cachetools               5.3.2
catalogue                2.0.10
certifi                  2024.2.2
charset-normalizer       3.3.2
chromadb                 0.3.26
click                    8.1.7
clickhouse-connect       0.7.0
coloredlogs              15.0.1
comm                     0.2.1
commonmark               0.9.1
cymem                    2.0.8
debugpy                  1.8.1
decorator                5.1.1
dill                     0.3.8
docker-pycreds           0.4.0
duckdb                   0.9.2
emoji                    2.10.1
exceptiongroup    

In [ ]:
! pip3 install allennlp

In [ ]:
! pip3 install numpy
! pip3 install pandas

! pip3 install torch

In [ ]:
! pip3 install nltk

In [ ]:
! pip3 install scikit-learn

In [ ]:
! pip3 install stanza

In [17]:
! pip3 install 'transformers<4.21' 

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached tokenizers-0.12.1-cp39-cp39-macosx_10_11_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.3.26 requires tokenizers>=0.13.2, but you have tokenizers 0.12.1 which is incompatible.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
! pip3 install tokenizers==0.11.1

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 3.7 MB 2.0 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.3.26 requires tokenizers>=0.13.2, but you have tokenizers 0.11.1 which is incompatible.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
# ! pip3 install typing-inspect==0.8.0 typing_extensions==4.5.0

In [ ]:
# ! pip3 install pydantic -U

In [ ]:
# ! pip3 install pydantic==1.10.11

In [4]:
import numpy as np
print("GOT HERE 1")
import pandas as pd
print("GOT HERE 2")
import json
print("GOT HERE 3")
import nltk
print("GOT HERE 4")
from data_utils import BuildPOS
print("GOT HERE 5")
from process_text import BuildVocab, TextToSentences
print("GOT HERE 6")
import torch.utils.data as data 
print("GOT HERE 7")
import time
print("GOT HERE 8")
import torch

2024-02-12 15:57:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


GOT HERE 1
GOT HERE 2
GOT HERE 3
GOT HERE 4
GOT HERE 5


2024-02-12 15:57:15 INFO: Loading these models for language: en (English):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2024-02-12 15:57:15 INFO: Using device: cpu
2024-02-12 15:57:15 INFO: Loading: tokenize
2024-02-12 15:57:15 INFO: Loading: mwt
2024-02-12 15:57:15 INFO: Loading: pos
2024-02-12 15:57:15 INFO: Done loading processors!
2024-02-12 15:57:15 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-02-12 15:57:16 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2024-02-12 15:57:16

GOT HERE 6
GOT HERE 7
GOT HERE 8


In [5]:
def pretty_print(epoch, loss):
    print('Epoch {} completed with loss {:.4f}'.format(epoch + 1, loss * (0.9 ** epoch)))

In [6]:
with open('data.json') as f:
    dataset = json.load(f)

In [7]:
vocab = BuildVocab()
start = time.time()
for i in range(len(dataset)):
    text = TextToSentences(dataset[i]['news'].lower())
    vocab.addText(text)
print('Vocabulary built in {:.4f} sec, with size: {}'.format(time.time() - start, vocab.n_words))

Vocabulary built in 0.9552 sec, with size: 6326


In [8]:
pos = BuildPOS()
start = time.time()
for i in range(len(dataset)):
    p = dataset[i]['pos']
    pos.addPOS(p)
print('POS built in {:.4f} sec, with size: {}'.format(time.time() - start, pos.n_pos))

POS built in 0.0008 sec, with size: 16


In [9]:
class StockPredictionDataset(data.Dataset):
    def __init__(self, dataset, vocab, pos):
        self.data = dataset
        self.vocab = vocab
        self.pos = pos
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        news_, pos_, stock_ = self.data[i]['news'], self.data[i]['pos'], self.data[i]['stock']
        news_ = torch.Tensor(self.process_sent(news_)).long()
        pos_ = torch.Tensor(self.process_pos(news_, pos_)).long()
        stock_ = np.array(stock_)
        y = int(np.sum((stock_[-1,:6] - stock_[-2,:6])) > 0) if len(stock_) >= 2 else 1
        stock_ = torch.Tensor(stock_)
        return news_, pos_, stock_, y
    
    def process_sent(self, sent):
        tokens = nltk.word_tokenize(sent.lower())
        out = []
        out.append(self.vocab.word2index['<SOS>'])
        out.extend([self.vocab.word2index[token] for token in tokens if token in self.vocab.word2index])
        out.append(self.vocab.word2index['<EOS>'])
        return out
    
    def process_pos(self, sent, pos):
        out = [self.pos.pos2index['<SOS>']]
        out.extend([self.pos.pos2index[token] for token in pos])
        if len(sent) <= (len(pos) + 1):
            pos = pos[:len(sent) - 2] 
        out.extend([self.pos.pos2index[pos[-1]] for _ in range(len(sent) - len(pos) - 2) ])
        out.append(self.pos.pos2index['<EOS>'])
        return out
            

In [10]:
def collate_fn(data):
    def merge(sequences, is_token):
        lengths = [len(seq) for seq in sequences]
        if is_token:
            padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        else:
            padded_seqs = torch.zeros(len(sequences), max(lengths), 120)
        for i, seq in enumerate(sequences):
            end = lengths[i]
            if end == 0:
                continue
            padded_seqs[i,:end] = seq[:]
        return padded_seqs, lengths
              
    news, pos, stock, y = [], [], [], []
    for i, (news_i, pos_i, stock_i, y_i) in enumerate(data):
        news.append(news_i)
        pos.append(pos_i)
        stock.append(stock_i)
        y.append(y_i)
    news.sort(key = lambda x: len(x), reverse=True)
    pos.sort(key = lambda x: len(x), reverse=True)
    stock.sort(key = lambda x: len(x), reverse=True)
    news, news_lengths = merge(news, True)
    pos, _ = merge(pos, True)
    stock, stock_lengths = merge(stock, False)
    y = torch.Tensor(y)
    return news, pos, stock, y, news_lengths, stock_lengths
    

In [11]:
dataset = StockPredictionDataset(dataset, vocab, pos)
dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [20]:
from model import MSSPM
import torch.nn as nn

In [21]:
word_emb_dim = 256
hidden_dim = 256
vocab_size = vocab.n_words
event_size = pos.n_pos
num_heads = 4
num_epochs = 10
criterion = nn.BCELoss()

In [ ]:
model = SSPM(vocab_size, word_emb_dim, hidden_dim, event_size, num_heads)
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

for e in range(num_epochs):
    for i, (news, pos, stock, y, sent_lengths, stock_lengths) in enumerate(dataloader):
        bsize = len(news)
        for i in range(bsize):
            if sent_lengths[i] == 0:
                sent_lengths[i] += 1
            elif stock_lengths[i] == 0:
                stock_lengths[i] += 1
        optimizer.zero_grad()
        
        """
        news = news.cuda()
        pos = pos.cuda()
        stock = stock.cuda()
        y = y.cuda().view(-1, 1)
        probs = model(news, pos, stock, sent_lengths, stock_lengths)
        loss = criterion(probs, y)
        loss.backward()
        optimizer.step()
    pretty_print(e, loss.item())   
    """   

In [22]:
word_emb_dim = 256
hidden_dim = 256
vocab_size = vocab.n_words
event_size = pos.n_pos
num_heads = 4
num_epochs = 10
eta = 0.5
criterion = nn.BCELoss()

In [23]:
model = MSSPM(vocab_size, word_emb_dim, hidden_dim, event_size, num_heads)
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
for e in range(num_epochs):
    for i, (news, pos, stock, y, sent_lengths, stock_lengths) in enumerate(dataloader):
        bsize = len(news)
        for i in range(bsize):
            if sent_lengths[i] == 0:
                sent_lengths[i] += 1
            elif stock_lengths[i] == 0:
                stock_lengths[i] += 1
        optimizer.zero_grad()
        news = news.cuda()
        pos = pos.cuda()
        stock = stock.cuda()
        y = y.cuda().view(-1, 1)
        probs, loss_crf = model(news, pos, stock, sent_lengths, stock_lengths)
        loss_bce = criterion(probs, y)
        loss = eta * loss_bce + (1 - eta) * loss_crf
        loss.backward()
        optimizer.step()
    pretty_print(e, loss.item())     

AssertionError: Attention hidden dimension is not divisible by the number of attention heads